In [26]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
import os


In [27]:
llm = ChatOpenAI( model="Qwen/Qwen2.5-7B-Instruct:together", api_key=os.environ["HF_TOKEN"], base_url="https://router.huggingface.co/v1" )


In [28]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], "The list of messages in the conversation, they can be HumanMessage or AIMessage", 
    add_messages] #reducer

In [29]:
def chat_node(state: ChatState):
    #take query from the user 
    messages = state['messages']
    #send to llm
    response = llm.invoke(messages)
    #store it in the state
    return {"messages": [response]}

In [30]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)


In [31]:
thread_id = '1'

while True:
    user_message = input("Type here: ")

    print('User:', user_message)

    if user_message.strip().lower() in ['exit', 'quit', 'bye']:
        break
    
    config = {'configurable': {'thread_id': thread_id}}
    response = chatbot.invoke({
        'messages': [HumanMessage(content=user_message)]}, config=config)  
    print('AI:', response['messages'][-1].content)

    

User: Hi my anme is Manas
AI: Hello Manas! Nice to meet you. How can I assist you today?
User: what is my name
AI: Your name is Manas.
User: my age is 26 nd i live in USA riht now
AI: Hello Manas! Nice to know you're 26 years old and currently living in the USA. How can I assist you today?
User: wHERE am i LIVING?
AI: You mentioned that you are currently living in the USA. If you have a specific city or state in mind, feel free to let me know, and I can provide more detailed information!
User: exit


In [32]:
chatbot.get_state(config)

StateSnapshot(values={'messages': [HumanMessage(content='Hi my anme is Manas', additional_kwargs={}, response_metadata={}, id='fd3f8c58-fa4e-4604-80d7-e49b0a527f10'), AIMessage(content='Hello Manas! Nice to meet you. How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 36, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'cached_tokens': 0}, 'model_provider': 'openai', 'model_name': 'Qwen/Qwen2.5-7B-Instruct-Turbo', 'system_fingerprint': None, 'id': 'oVXVHLn-4YNCb4-9c6b265f995a8205', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c153d-3724-7f13-baa2-d28261601e64-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 36, 'output_tokens': 17, 'total_tokens': 53, 'input_token_details': {}, 'output_token_details': {}}), HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}, id='95221b8e-a67f-457d-bed1-9d